In [2]:
import pandas as pd
import os

In [3]:
home = os.path.expanduser("~")
ar = pd.read_csv(home + "/data/core-rules/dump-2025-09-09.csv")
ar.head(5)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status
0,CDISC.SDTMIG.CG0320,CG0320,'--TESTCD does not exist in custom Findings do...,Raise an error when variable --TESTCD is missi...,SDTMIG,"3.4, 3.2, 3.3","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Draft
1,NaN,FB5401,POOLID must have a corresponding set of record...,Generate an issue when POOLID is not in the un...,"SENDIG, SENDIG-GENETOX, SENDIG-DART","3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""INTERVENTIONS"",""FINDIN...",Fully Executable,Draft
2,NaN,FB1115,The set of parameters without conditions liste...,Check whether all values in the list are popul...,SENDIG,"3.0, 3.1, 3.1.1","{""Classes"":{""Include"":[""TRIAL DESIGN""]},""Domai...",Fully Executable,Draft
3,CORE-000857,DDF00141,The planned sex is not specified using the Sex...,A planned sex must be specified using the Sex ...,USDM,"3.0, 4.0","{""Entities"":{""Include"":[""StudyDesignPopulation...",Fully Executable,Draft
4,CORE-000861,DDF00179,The administrable dose form is not specified a...,An administrable dose form must be specified a...,USDM,4.0,"{""Entities"":{""Include"":[""AdministrableProduct""]}}",Fully Executable,Draft


In [4]:
ar['std'] = ar['Standard Name'].apply(lambda x: x.split(', ') if isinstance(x, str) else None)
ar['rids'] = ar['CDISC Rule ID'].apply(lambda x: x.split(', ') if isinstance(x, str) else None)
ar.head(5)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status,std,rids
0,CDISC.SDTMIG.CG0320,CG0320,'--TESTCD does not exist in custom Findings do...,Raise an error when variable --TESTCD is missi...,SDTMIG,"3.4, 3.2, 3.3","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Draft,[SDTMIG],[CG0320]
1,NaN,FB5401,POOLID must have a corresponding set of record...,Generate an issue when POOLID is not in the un...,"SENDIG, SENDIG-GENETOX, SENDIG-DART","3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""INTERVENTIONS"",""FINDIN...",Fully Executable,Draft,"[SENDIG, SENDIG-GENETOX, SENDIG-DART]",[FB5401]
2,NaN,FB1115,The set of parameters without conditions liste...,Check whether all values in the list are popul...,SENDIG,"3.0, 3.1, 3.1.1","{""Classes"":{""Include"":[""TRIAL DESIGN""]},""Domai...",Fully Executable,Draft,[SENDIG],[FB1115]
3,CORE-000857,DDF00141,The planned sex is not specified using the Sex...,A planned sex must be specified using the Sex ...,USDM,"3.0, 4.0","{""Entities"":{""Include"":[""StudyDesignPopulation...",Fully Executable,Draft,[USDM],[DDF00141]
4,CORE-000861,DDF00179,The administrable dose form is not specified a...,An administrable dose form must be specified a...,USDM,4.0,"{""Entities"":{""Include"":[""AdministrableProduct""]}}",Fully Executable,Draft,[USDM],[DDF00179]


In [5]:
unique_strings = list(pd.unique(ar['std'].explode().dropna()))
print(unique_strings)

# BEWARE - THIS IS A MANUALLY CREATED LIST FROM THE OUPUTS OF THE NEXT CELL
relevant_stds = {'SDTMIG', 'ADaMIG', 'Internal Standard', 'ADaMIG-MD', 'ADTTE', 'ADAMIG', 'Define-XML'}


['SDTMIG', 'SENDIG', 'SENDIG-GENETOX', 'SENDIG-DART', 'USDM', 'ADaMIG', 'SENDIG-AR', 'TIG', 'ADaMIG-MD', 'ADTTE', 'OCCDS', 'ADAMIG', 'Model v2.1', 'Define-XML', 'SENDID-GENETOX']


In [6]:
counts = (
    ar["std"]
      .dropna()
      .map(lambda x: set(x))   # ensure per-row uniqueness
      .explode()
      .value_counts()
)

print(counts)

std
SDTMIG            1120
SENDIG             669
SENDIG-DART        658
SENDIG-AR          554
TIG                360
SENDIG-GENETOX     309
USDM               178
ADaMIG             103
ADaMIG-MD           36
OCCDS               14
ADAMIG               4
SENDID-GENETOX       2
ADTTE                1
Model v2.1           1
Define-XML           1
Name: count, dtype: int64


In [7]:
mask = ar["std"].apply(
    lambda x: bool(relevant_stds.intersection(x)) if isinstance(x, (list, set, tuple)) else False
)

filtered_by_std = ar[mask]
filtered_by_std.head(10)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status,std,rids
0,CDISC.SDTMIG.CG0320,CG0320,'--TESTCD does not exist in custom Findings do...,Raise an error when variable --TESTCD is missi...,SDTMIG,"3.4, 3.2, 3.3","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Draft,[SDTMIG],[CG0320]
12,CDISC.ADaMIG.AD0817,817,SUBJIDN is not unique for a given value of SUBJID,NaN,ADaMIG,1.3,"{""Subclasses"":{""Include"":[""POPULATION PHARMACO...",Fully Executable,Draft,[ADaMIG],[817]
13,CORE-000867,FB1501,Text variables in submitted datasets contain s...,Text variables in submitted datasets should no...,"SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.1, 1.2, 1.0","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDI...",[FB1501]
14,CORE-000866,FB3210,'--DTC is greater than --ENDTC,Date/Time of Specimen Collection (--DTC) must ...,"SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, S...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""FINDINGS""]},""Domains"":...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, ...",[FB3210]
15,CORE-000865,FB4405,'--ELTM must be populated when --TPTREF is pop...,'--ELTM must be populated when --TPTREF is pop...,"SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.1, 1.2, 1.0","{""Classes"":{""Include"":[""FINDINGS"",""EVENTS"",""IN...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDI...",[FB4405]
16,NaN,FB1502,Text variables in submitted datasets contain ...,Value does not contain any alphanumeric charac...,"SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.1, 1.2, 1.0","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Draft,"[SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDI...",[FB1502]
17,CDISC.ADaMIG.AD0813,813,USUBJID is not unique for a given value of USU...,NaN,ADaMIG,1.3,"{""Subclasses"":{""Include"":[""POPULATION PHARMACO...",Fully Executable,Draft,[ADaMIG],[813]
18,NaN,FB3102,'--STRESC is numeric but --STRESN is not equal...,"'--STRESC is populated with a numeric value, b...","SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.1, 1.2, 1.0","{""Classes"":{""Include"":[""FINDINGS""]},""Domains"":...",Fully Executable,Draft,"[SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDI...",[FB3102]
19,CORE-000864,FB3206,The End Date/Time of Observation (--ENDTC) is ...,End Study Date/Time (--ENDTC) variable should ...,"SDTMIG, SENDIG, SENDIG-AR, SENDIG-DART, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""INTERVENTIONS"",""EVENTS...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-AR, SENDIG-DART, SENDI...",[FB3206]
20,CORE-000863,FB3101,"'--STRESC is populated with a numeric value, b...",When --STRESC is populated with a numeric valu...,"SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.1, 1.2, 1.0","{""Classes"":{""Include"":[""FINDINGS""]},""Domains"":...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDI...",[FB3101]


In [8]:
# drop all old rules, they start with either "sd" or "ct"

mask = filtered_by_std["rids"].apply(
    lambda xs: not any(
        isinstance(s, str) and s.lower().startswith(("sd", "ct"))
        for s in xs
    ) if isinstance(xs, (list, tuple, set)) else True
)
filtered_by_rid = filtered_by_std[mask]
print(filtered_by_std.shape, filtered_by_rid.shape)

(1240, 11) (760, 11)


In [9]:
counts = (
    filtered_by_rid["std"]
      .dropna()
      .map(lambda x: set(x))   # ensure per-row uniqueness
      .explode()
      .value_counts()
)

print(counts)

std
SDTMIG            640
TIG               280
ADaMIG            103
SENDIG             95
SENDIG-DART        94
SENDIG-GENETOX     93
SENDIG-AR          62
ADaMIG-MD          36
OCCDS              14
ADAMIG              4
ADTTE               1
Model v2.1          1
Define-XML          1
USDM                1
Name: count, dtype: int64


In [10]:
filtered_by_rid.head(200)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status,std,rids
0,CDISC.SDTMIG.CG0320,CG0320,'--TESTCD does not exist in custom Findings do...,Raise an error when variable --TESTCD is missi...,SDTMIG,"3.4, 3.2, 3.3","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Draft,[SDTMIG],[CG0320]
12,CDISC.ADaMIG.AD0817,817,SUBJIDN is not unique for a given value of SUBJID,NaN,ADaMIG,1.3,"{""Subclasses"":{""Include"":[""POPULATION PHARMACO...",Fully Executable,Draft,[ADaMIG],[817]
13,CORE-000867,FB1501,Text variables in submitted datasets contain s...,Text variables in submitted datasets should no...,"SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.1, 1.2, 1.0","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDI...",[FB1501]
14,CORE-000866,FB3210,'--DTC is greater than --ENDTC,Date/Time of Specimen Collection (--DTC) must ...,"SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, S...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""FINDINGS""]},""Domains"":...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, ...",[FB3210]
15,CORE-000865,FB4405,'--ELTM must be populated when --TPTREF is pop...,'--ELTM must be populated when --TPTREF is pop...,"SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDIG...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.1, 1.2, 1.0","{""Classes"":{""Include"":[""FINDINGS"",""EVENTS"",""IN...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-DART, SENDIG-AR, SENDI...",[FB4405]
...,...,...,...,...,...,...,...,...,...,...,...
420,CORE-000046,"CG0519, TIG0614",ARMNRS is missing when ARM value is missing,Verify ARMNRS is not null when ARM is null,"SDTMIG, TIG","3.4, 3.3, 1.0","{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Fully Executable,Published,"[SDTMIG, TIG]","[CG0519, TIG0614]"
421,CORE-000024,"CG0082, TIG0347",'--BODSYS is not empty and --BDSYCD is empty,"Raise an error if --BODSYS is not empty, --BDS...","SDTMIG, TIG","3.4, 3.3, 3.2, 1.0","{""Classes"":{""Include"":[""EVENTS""]},""Domains"":{""...",Fully Executable,Published,"[SDTMIG, TIG]","[CG0082, TIG0347]"
422,CORE-000045,"CG0517, TIG0612",ARMNRS value is missing when ARMCD value is mi...,Verify ARMNRS is not null when ARMCD is null,"SDTMIG, TIG","3.4, 3.3, 1.0","{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Fully Executable,Published,"[SDTMIG, TIG]","[CG0517, TIG0612]"
423,CORE-000012,"CG0040, TIG0319",AEOCCUR is present in AE dataset.,Raise an error when AEOCCUR exists in AE dataset.,"SDTMIG, TIG","3.4, 3.3, 3.2, 1.0","{""Classes"":{""Include"":[""EVENTS""]},""Domains"":{""...",Fully Executable,Published,"[SDTMIG, TIG]","[CG0040, TIG0319]"


In [11]:
# get SDTM rules
sdtm = filtered_by_rid[filtered_by_rid["std"].apply(lambda x: "SDTMIG" in x)].copy()
print('sdtm', sdtm.shape)

# get ADaM rules

wanted = {'ADaMIG', 'ADaMIG-MD', 'ADTTE', 'ADAMIG'}
mask = filtered_by_rid["std"].apply(
    lambda xs: (
        any(s in wanted for s in xs) if isinstance(xs, (list, tuple, set))
        else isinstance(xs, str) and xs in wanted
    )
)
adam = filtered_by_rid[mask].copy()
print('adam', adam.shape)

# get Define-XML rules
define = filtered_by_rid[filtered_by_rid["std"].apply(lambda x: "Define-XML" in x)].copy()
print('define', define.shape)

sdtm (640, 11)
adam (121, 11)
define (1, 11)


In [12]:
sdtm.to_csv(home + "/Code/verisian-cdisc/tests/resources/rules/sdtm_rules.csv", index=False, header=True)
adam.to_csv(home + "/Code/verisian-cdisc/tests/resources/rules/adam_rules.csv", index=False, header=True)
define.to_csv(home + "/Code/verisian-cdisc/tests/resources/rules/define_rules.csv", index=False, header=True)